In [1]:
# setting device on GPU if available, else CPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

# Additional Info when using cuda
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory Usage:")
    print("Allocated:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), "GB")
    print("Cached:   ", round(torch.cuda.memory_reserved(0) / 1024**3, 1), "GB")

Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from functools import partial
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

## test rendering

In [4]:
def findAllFile(base):
    """
    Recursively find all files in the specified directory.

    Args:
        base (str): The base directory to start the search.

    Returns:
        list: A list of file paths found in the directory and its subdirectories.
    """
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path

In [5]:
from utils.motion_processing.hml_process import recover_from_ric, recover_root_rot_pos
import utils.vis_utils.plot_3d_global as plot_3d
import matplotlib.pyplot as plt

In [6]:
from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser

from configs.config import cfg, get_cfg_defaults
from configs.config_streaming import get_cfg_defaults as strm_get_cfg_defaults


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
gen_cfg = strm_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_streaming/motion_streaming.yaml")
gen_cfg.freeze()

In [8]:

body_cfg = get_cfg_defaults()
body_cfg.merge_from_file(gen_cfg.vqvae.body_config)
# left_cfg = get_cfg_defaults()
# left_cfg.merge_from_file(gen_cfg.vqvae.left_hand_config)
# right_cfg = get_cfg_defaults()
# right_cfg.merge_from_file(gen_cfg.vqvae.right_hand_config)

In [9]:
from core.models.resnetVQ.vqvae import HumanVQVAE


In [10]:
# left_hand_model = HumanVQVAE(left_cfg.vqvae).to(device).eval()
# left_hand_model.load(os.path.join(left_cfg.output_dir, "vqvae_motion.pt"))

# right_hand_model = HumanVQVAE(right_cfg.vqvae).to(device).eval()
# right_hand_model.load(os.path.join(right_cfg.output_dir, "vqvae_motion.pt"))

body_model = HumanVQVAE(body_cfg.vqvae).to(device).eval()
body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))



### VQVAE

In [10]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset as load_dataset_vq
from core.datasets.vq_dataset import simple_collate as simple_collate_vq


In [11]:
def get_decoded(model , motion):
    encs = []
    inds = []
    for i in range(0 , motion.shape[1] , 120 ):
        if i + 240 >= motion.shape[1]:
            enc_b = model(motion[: , i:, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
            break
        else:
            enc_b = model(motion[: , i:i + 120, :].to(device))
            encs.append(enc_b.decoded_motion)
            inds.append(enc_b.indices)
    return torch.cat(encs , 1), torch.cat(inds , -1)

In [12]:
condition_provider2 = ConditionProvider(
            motion_rep=MotionRep("body"),
            motion_padding="max_length",

        )

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [47]:
dataset_names_default = [
    # "animation",
    # "humanml",
    # "perform",
    # "GRAB",
    "idea400",
    # "humman",
    # "beat",
    # "game_motion",
    # "music",
    # "aist",
    # "fitness",
    # "moyo",
    # "choreomaster",
    # "dance",
    # "kungfu",
    # "EgoBody",
    # "HAA500",
]

In [204]:
body_cfg = get_cfg_defaults()
body_cfg.merge_from_file(gen_cfg.vqvae.body_config)

In [205]:
body_cfg.dataset.hml_rep = "gprv"
body_cfg.dataset.window_size = -1

In [206]:
test_ds, _, _ = load_dataset_vq(
            dataset_names=dataset_names_default,
            dataset_args=body_cfg.dataset,
            split="render",
        )

Total number of motions idea400: 2


In [207]:
dl = torch.utils.data.DataLoader(
            test_ds,
            batch_size=1,
            sampler=None,
            shuffle=False,
            collate_fn=partial(simple_collate_vq , conditioner = condition_provider2),
            
        )

In [208]:
for batch in tqdm(dl):
    continue


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 106.76it/s]


In [233]:
mot = batch["motion"][0][0]

In [89]:
mot.shape

torch.Size([239, 259])

In [69]:
# 

In [144]:
# mot[:,1:4] = 0

In [229]:
from utils.motion_processing.quaternion import qinv, qrot, qmul, quaternion_to_cont6d, qeuler, euler_to_quaternion, euler2quat
import utils.rotation_conversions as geometry
from core.datasets.kinematics import getSkeleton


In [194]:
mot[:,0] = 0

In [234]:
Motion = test_ds.datasets[0].toMotion(mot)
r_rot_quat, r_pos = test_ds.datasets[0].mot2pos.recover_root_rot_pos(Motion)

In [215]:
r_pos[:] = 0

In [179]:
inv_r_rot_quat = qinv(r_rot_quat)


In [180]:
inv_r_rot_quat = inv_r_rot_quat[:,None,:].repeat(1, 21 , 1)

In [181]:
cont6d_params = Motion.rotations

In [182]:
quat_params = geometry.matrix_to_quaternion(geometry.rotation_6d_to_matrix(cont6d_params.reshape(239 , -1, 6)))

In [183]:
inv_root_rot_quat_params = qmul(quat_params , inv_r_rot_quat)
inv_root_rot_6d_params = quaternion_to_cont6d(inv_root_rot_quat_params).reshape(-1 , 126)

In [216]:
def remove_y_rotation(A, B):
    """Remove rotation around the y-axis from A using B."""
    _, _, yaw = quaternion_to_euler(B)
    B_yaw_only = euler_to_quaternion(0, 0, yaw)
    B_conjugate = quaternion_conjugate(B_yaw_only)
    A_rotated = quaternion_multiply(A, B_conjugate)
    return A_rotated


A_rotated: [2.82842712 4.24264069 1.41421356 1.41421356]


In [236]:
e = qeuler(r_rot_quat , "xyz")
e[: , [0,2]] = 0
q = euler2quat(e, "xyz", deg=True)

In [189]:
data = test_ds.datasets[0].toMotion(inv_root_rot_6d_params , hml_rep = "r")

In [185]:
data.root_params = torch.zeros((data.rotations.shape[:-1] + (4,)))

In [188]:
skeleton = getSkeleton(
            os.path.join(test_ds.datasets[0].mot2pos.data_root, "motion_data/000021_pos.npy"),
            motion_rep=data.motion_rep.value,
        )

In [ ]:
cont6d_params = data.rotations
cont6d_params = torch.cat([r_rot_cont6d, cont6d_params], dim=-1)
cont6d_params = cont6d_params.view(-1, joints_num, 6)

In [ ]:
positions = skeleton.forward_kinematics_cont6d(cont6d_params, r_pos )

In [191]:
test_ds.datasets[0].render_hml(data , save_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/idea400_r.gif" )


In [212]:
Motion = test_ds.datasets[0].toMotion(mot)
Motion.hml_rep = "gr"
test_ds.datasets[0].render_hml(Motion , save_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/idea400_r.gif" )


In [217]:
import numpy as np

def quaternion_multiply(q1, q2):
    """Multiply two quaternions."""
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2
    return np.array([w, x, y, z])

def quaternion_conjugate(q):
    """Compute the conjugate of a quaternion."""
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

def euler_to_quaternion(roll, pitch, yaw):
    """Convert Euler angles to quaternion."""
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)

    w = cy * cp * cr + sy * sp * sr
    x = cy * cp * sr - sy * sp * cr
    y = sy * cp * cr + cy * sp * sr
    z = cy * sp * cr - sy * cp * sr

    return np.array([w, x, y, z])

def quaternion_to_euler(q):
    """Convert quaternion to Euler angles."""
    w, x, y, z = q
    ysqr = y * y

    # roll (x-axis rotation)
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    roll = np.arctan2(t0, t1)

    # pitch (y-axis rotation)
    t2 = +2.0 * (w * y - z * x)
    t2 = np.where(t2 > 1.0, 1.0, t2)
    t2 = np.where(t2 < -1.0, -1.0, t2)
    pitch = np.arcsin(t2)

    # yaw (z-axis rotation)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    yaw = np.arctan2(t3, t4)

    return roll, pitch, yaw

def remove_y_rotation(A, B):
    """Remove rotation around the y-axis from A using B."""
    _, _, yaw = quaternion_to_euler(B)
    B_yaw_only = euler_to_quaternion(0, 0, yaw)
    B_conjugate = quaternion_conjugate(B_yaw_only)
    A_rotated = quaternion_multiply(A, B_conjugate)
    return A_rotated

# Example usage:
A = np.array([1, 2, 3, 4])  # Example quaternion A
B = euler_to_quaternion(0, np.pi / 2, 0)  # Example quaternion B with rotation around y-axis
A_rotated = remove_y_rotation(A, B)
print("A_rotated:", A_rotated)


A_rotated: [2.82842712 4.24264069 1.41421356 1.41421356]


In [71]:
# import imageio
# import numpy as np    
# from PIL import Image
# #Create reader object for the gif
# gif1 = imageio.get_reader("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/idea400_rv.gif")
# gif2 = imageio.get_reader("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/idea400_grv.gif")

# #If they don't have the same number of frame take the shorter
# number_of_frames = min(gif1.get_length(), gif2.get_length()) - 1

# #Create writer object
# new_gif = imageio.get_writer("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/idea400_comb.gif")

# for frame_number in tqdm(range(number_of_frames)):
#     img1 = gif1.get_next_data()
#     img2 = gif2.get_next_data()
#     #here is the magic
#     new_image = np.hstack((img1, img2))
#     new_gif.append_data(new_image)

# print("closing")


# gif1.close()
# gif2.close()    
# new_gif.close()

100%|█████████████████████████████████████████| 238/238 [00:22<00:00, 10.42it/s]


closing


In [18]:
# errors = []
# for batch in tqdm(dl):
#     sve = os.path.join("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body" , batch["names"][0] )
#     if os.path.exists(sve):
#         continue
#     os.makedirs(os.path.dirname(sve) , exist_ok=True)
#     try:
#         enc_b, inb = get_decoded(body_model , batch["motion"][0].to(device))
#         np.save(sve , inb.cpu().numpy())
#     except:
#         errors.append(batch["names"][0])
    

100%|████████████████████████████████████████████████████████████████████████████████| 19984/19984 [1:02:28<00:00,  5.33it/s]


In [19]:
len(errors)

0

In [7]:
fles = findAllFile("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/")

In [13]:
err = []
for i in fles:
    if np.load(i).shape[-1] < 10:
        err.append(os.path.basename(i))
        print(i, np.load(i).shape)

/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/006859.npy (1, 9)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/013180.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/M005024.npy (1, 9)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/M006988.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/008362.npy (1, 8)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/M011994.npy (1, 8)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/010908.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/009136.npy (1, 7)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/013310.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/M006950.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/003216.npy (1, 6)
/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/011011.npy (1, 8)
/srv/hays-lab/scratch/sanisetty3/mot

In [21]:
np.load("/srv/hays-lab/scratch/sanisetty3/motionx/indices/body/humanml/M012813.npy").shape

(1, 74)

In [15]:
"005485.npy" in err

False

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 115.37it/s]


In [26]:
for batch in tqdm(dl):
    test_ds.datasets[0].render_hml(batch["motion"][0][0][:300] , save_path = f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/true_all/{os.path.basename(batch['names'][0])}.gif" )

100%|████████████████████████████████████████████████████████████████████████████████████████| 33/33 [10:38<00:00, 19.34s/it]


In [38]:
batch.keys()

dict_keys(['names', 'motion'])

In [29]:
batch["names"]

array(['idea400/subset_0056/Walking_And_Snapping_At_The_Same_Time.npy'],
      dtype='<U61')

In [28]:
batch["motion"][0].shape

torch.Size([1, 120, 196])